In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
train_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')
test_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [ ]:
labels=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
labels

In [ ]:
sample=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sample

In [ ]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
path0='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-1.dcm'
img1=load_dicom(path0)
img2=cv2.resize(img1,(9,9)) 
print(img1.shape)
print(img2.shape)

In [ ]:
train_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
trainset=[]
trainlabel=[]
trainidt=[]
for i in tqdm(range(len(labels))):
    idt=labels.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:]
    path=os.path.join(train_dir,idt2,'T1wCE')              
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        trainset+=[image]
        trainlabel+=[labels.loc[i,'MGMT_value']]
        trainidt+=[idt]

In [ ]:
test_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
testset=[]
testidt=[]
for i in tqdm(range(len(sample))):
    idt=sample.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:]
    path=os.path.join(test_dir,idt2,'T1wCE')               
    for im in os.listdir(path):   
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        testset+=[image]
        testidt+=[idt]

In [ ]:
from tensorflow.keras.utils import to_categorical
y0=np.array(trainlabel)
Y_train=to_categorical(y0)
X_train=np.array(trainset)
X_test=np.array(testset)

In [ ]:
model = Sequential()
model.add(Conv2D(64,(4,4),input_shape = (9,9,1),activation = 'relu'))
model.add(Conv2D(32,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
his = model.fit(X_train, Y_train, validation_split=0.2, epochs=100, batch_size=64, verbose=2)

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
sample

In [ ]:
y_pred=model.predict(X_test)
pred=np.argmax(y_pred,axis=1)
result=pd.DataFrame(testidt)
result[1]=pred
result.columns=['BraTS21ID','MGMT_value']
result2=result.groupby('BraTS21ID',as_index=False).mean()
result2

In [ ]:
result2['BraTS21ID']=sample['BraTS21ID']
result2['MGMT_value']=result2['MGMT_value'].apply(lambda x:round(x*10)/10)
result2.to_csv('submission.csv',index=False)
result2